#DSen2 Superresolution 
*Johannes Mast and Matjaž Krebs in collaboration with Žiga Kokalj*
 
 
*14.07.2020*

## Introduction
 
 
This script is an implementation of *Super-resolution of Sentinel-2 images: Learning a globally applicable deep neural network* [[Link]](https://arxiv.org/abs/1803.04271) by Charis Lanaras, José Bioucas-Dias, Silvano Galliani, Emmanuel Baltsavias and Konrad Schindler. The code for preprocessing, the model, and the model weights were made available online on [github](https://github.com/lanha/DSen2) and credit for developing and testing the model and the preprocessing goes to them. 
 
Johannes Mast, the [author of this script](https://github.com/JohMast), adapted the code under supervision of Žiga Kokalj for operational use and Matjaž Krebs adapted it further for his diploma thesis to train the weights specifically for the territory of Slovenia in order to test the viability of the supperesolved datasets for drought detection. In doing so we removed certain functionalities removed, specifically query options, and thereby simplified the code. Further, changes allow the functions to be applied to a list of files.

The published code is suitable to be called via command line. The functions were adapted so they can be used as python functions within a script such as the present one.

## Overview

For a demonstration of the inner workings of the DSen2 code, we refer to the [demonstration script](https://github.com/JohMast/DSen2_Implementation/blob/master/Demonstration_of_DSen2_Superresolution_.ipynb) which can be executed in google colab on a VM with the only requirements being a Google Drive with about 2GB available space, and an account on the [Copernicus Open Access Hub](https://scihub.copernicus.eu/dhus/).

In the present script, we will merely see how to use the high-level functions which, as mentioned previously, are simplified but faster. Unlike the demonstration script, this script is tied to the contents of the [github repository](https://github.com/JohMast/DSen2_Implementation) and relies on the files available there. It is intended to be run on google colab but should also work just fine on other platforms that have the required Python 3 packages. Notably, as of the time that this script was created, Tensorflow 1 is required.




## The Script

This script consists of three parts:

*   First, we will do a quick **Setup**. This is necessary on google colab and the steps may vary on other platforms. Afterwards, we have access to two alternatives:
*   **A: Training**: This consists of preprocessing of the training data and the training of the model itself, either from scratch or from a previously created weights file.
*   **B: Prediction**: This is the superresolution. We predict on our images using a previously created weights file.

Since weights files are provided for all network types, it is not necessary to perform **A** before **B**. However, it might make sense to tune the network to our images before the prediction.



### Setup



####Directory Structure
For the purpose of this script, we will assume that the following file structure already exists:

* DSen2OP
    * Data
        * Inputs
            * `Input Sentinel-2 Files, zipped`
        * Outputs
        * Training
    * DSen2
        * models
            * `The model weights`
        * python
            * `All python files`

However, this is only a suggestion. You can adjust the path names in the script to your own directory structure. Only the structure within the DSen2 folder should be kept intact.

#### Connecting to Google Drive

Every time the notebook runtime is started, it must be reconnected to the Google Drive.
Therefore, this must be done at the beginning of every script. An authorisation code is required as an input by the user.

In [ ]:
%%capture
from google.colab import drive
from importlib import reload

drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········


#### Navigating to the folder

We usually have a main working directory from which we operate. For us, it is called `DSen2OP`. We navigate to it.

In [ ]:

#prvotno v zvezku
%cd "gdrive/My Drive/DSen2OP"

/content/gdrive/My Drive/DSen2OP


#### Selecting  Tensorflow Version

Colab VMs offer different versions of tensorflow - here, we need tensorflow 1.15.0, the default as of 27.03.2020 is tensorflow 2.

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

TensorFlow 1.x selected.
Tensorflow version 1.15.2


### Pridobivanje podatkov




In [ ]:
####prenos posnetkov
#programski paketi
%%capture
!pip install rasterio
!pip install geopandas
!pip install descartes
!pip install gdal
!pip install sentinelsat
!pip install geojson
!pip install shapely
!pip install Pillow

In [ ]:
n_dl_images=30

In [ ]:
#območje
lon_min=13.5
lon_max=16
lat_min= 45.5
lat_max= 46.5

In [ ]:
# povezava na api
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
import getpass
# connect to the API
api = SentinelAPI(getpass.getpass(prompt="Please enter Copernicus Open Access Hub username"),
                  getpass.getpass(prompt="Please enter Copernicus Open Access Hub password"),
                  'https://scihub.copernicus.eu/dhus')

Please enter Copernicus Open Access Hub username··········
Please enter Copernicus Open Access Hub password··········


In [ ]:
#poligon AOI

aoi = 'POLYGON((%s %s,%s %s,%s %s,%s %s,%s %s))'  %(lon_min,lat_min,lon_min,lat_max,lon_max,lat_max,lon_max,lat_min,lon_min,lat_min)
import plotly.graph_objects as go
fig = go.Figure(go.Scattermapbox(
    fill = "toself",text="Query Area",
    lon = [lon_min, lon_max, lon_max, lon_min],
    lat = [lat_max, lat_max, lat_min, lat_min],
    marker = { 'size': 10, 'color': "red" }))
fig.update_layout(
     title="AOI for the Query",font=dict(family="Arial",size=18,),
     mapbox = {
        'style': "stamen-terrain",
        'center': {'lon': ((lon_min+lon_max)/2), 'lat': ((lat_min+lat_max)/2) },
        'zoom': 4},
    showlegend = False)
fig.show()


In [ ]:
#query
# search by polygon, time, and SciHub query keywords
products = api.query(aoi,
                     date=('20190301', date(2019, 5, 31)),
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 10))

In [ ]:
# convert to Pandas DataFrame
products_df = api.to_dataframe(products)

# sort and limit to first n_dl_images sorted products
products_df_sorted = products_df.sort_values(['cloudcoverpercentage', 'ingestiondate',], ascending=[True, True])
products_df_sorted = products_df_sorted.head(n_dl_images)
products_df_sorted


,title,link,link_alternative,link_icon,summary,beginposition,endposition,ingestiondate,orbitnumber,relativeorbitnumber,cloudcoverpercentage,highprobacloudspercentage,mediumprobacloudspercentage,snowicepercentage,vegetationpercentage,waterpercentage,notvegetatedpercentage,unclassifiedpercentage,gmlfootprint,format,instrumentshortname,instrumentname,footprint,s2datatakeid,platformidentifier,orbitdirection,platformserialidentifier,processingbaseline,processinglevel,producttype,platformname,size,filename,level1cpdiidentifier,identifier,uuid,datatakesensingstart,sensoroperationalmode,tileid,hv_order_tileid,granuleidentifier,datastripidentifier
900db3ef-ff9b-4f0a-b0e5-07f5eea720f8,S2A_MSIL1C_20190302T100021_N0207_R122_T33TUL_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-03-02T10:00:21.024Z, Instrument: MS...",2019-03-02 10:00:21.024,2019-03-02 10:00:21.024,2019-03-10 04:43:36.592,19281,122,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<gml:Polygon srsName=""http://www.opengis.net/g...",SAFE,MSI,Multi-Spectral Instrument,MULTIPOLYGON (((12.460785793001175 45.03702311...,GS2A_20190302T100021_019281_N02.07,2015-028A,DESCENDING,Sentinel-2A,02.07,Level-1C,S2MSI1C,Sentinel-2,684.60 MB,S2A_MSIL1C_20190302T100021_N0207_R122_T33TUL_2...,S2A_OPER_MSI_L1C_TL_SGS__20190302T134355_A0192...,S2A_MSIL1C_20190302T100021_N0207_R122_T33TUL_2...,900db3ef-ff9b-4f0a-b0e5-07f5eea720f8,2019-03-02 10:00:21.024,INS-NOBS,33TUL,TL33U,S2A_OPER_MSI_L1C_TL_SGS__20190302T134355_A0192...,S2A_OPER_MSI_L1C_DS_SGS__20190302T134355_S2019...
74499ca0-f454-4af3-bbea-4d4714f3e52b,S2A_MSIL1C_20190312T100021_N0207_R122_T33TUL_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-03-12T10:00:21.024Z, Instrument: MS...",2019-03-12 10:00:21.024,2019-03-12 10:00:21.024,2019-03-12 18:57:23.067,19424,122,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<gml:Polygon srsName=""http://www.opengis.net/g...",SAFE,MSI,Multi-Spectral Instrument,MULTIPOLYGON (((13.834778102586263 46.01494574...,GS2A_20190312T100021_019424_N02.07,2015-028A,DESCENDING,Sentinel-2A,02.07,Level-1C,S2MSI1C,Sentinel-2,17.02 MB,S2A_MSIL1C_20190312T100021_N0207_R122_T33TUL_2...,S2A_OPER_MSI_L1C_TL_MPS__20190312T120945_A0194...,S2A_MSIL1C_20190312T100021_N0207_R122_T33TUL_2...,74499ca0-f454-4af3-bbea-4d4714f3e52b,2019-03-12 10:00:21.024,INS-NOBS,33TUL,TL33U,S2A_OPER_MSI_L1C_TL_MPS__20190312T120945_A0194...,S2A_OPER_MSI_L1C_DS_MPS__20190312T120945_S2019...
c08ad696-7537-482b-aa70-7bd6cc7fa604,S2A_MSIL1C_20190312T100021_N0207_R122_T33TUL_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-03-12T10:00:21.024Z, Instrument: MS...",2019-03-12 10:00:21.024,2019-03-12 10:00:21.024,2019-03-12 21:12:00.546,19424,122,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<gml:Polygon srsName=""http://www.opengis.net/g...",SAFE,MSI,Multi-Spectral Instrument,MULTIPOLYGON (((12.460785793001175 45.03702311...,GS2A_20190312T100021_019424_N02.07,2015-028A,DESCENDING,Sentinel-2A,02.07,Level-1C,S2MSI1C,Sentinel-2,682.67 MB,S2A_MSIL1C_20190312T100021_N0207_R122_T33TUL_2...,S2A_OPER_MSI_L1C_TL_SGS__20190312T134427_A0194...,S2A_MSIL1C_20190312T100021_N0207_R122_T33TUL_2...,c08ad696-7537-482b-aa70-7bd6cc7fa604,2019-03-12 10:00:21.024,INS-NOBS,33TUL,TL33U,S2A_OPER_MSI_L1C_TL_SGS__20190312T134427_A0194...,S2A_OPER_MSI_L1C_DS_SGS__20190312T134427_S2019...
c27b63b5-2baf-4cdd-843c-2fb2537d88a2,S2B_MSIL1C_20190324T095029_N0207_R079_T33TVL_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-03-24T09:50:29.024Z, Instrument: MS...",2019-03-24 09:50:29.024,2019-03-24 09:50:29.024,2019-03-24 17:29:36.716,10687,79,0.000000,NaN,N

In [ ]:
#izpis v excel
import pandas as pd
df = products_df_sorted

writer = pd.ExcelWriter('2018.xlsx', mode= 'a')
df.to_excel(writer, sheet_name='sep-dec30')
writer.save()


In [ ]:

# download sorted and reduced products
#%%capture 

api.download_all(products_df_sorted.index,"Data/novo/")




Downloading:   0%|          | 0.00/1.17G [00:00<?, ?B/s]
Downloading:   1%|          | 7.34M/1.17G [00:02<10:30, 1.85MB/s]
Downloading:   0%|          | 1.05M/1.08G [00:01<22:44, 794kB/s]
Downloading:   1%|          | 9.44M/1.17G [00:02<08:29, 2.28MB/s]
Downloading:   1%|          | 12.6M/1.17G [00:02<06:27, 2.99MB/s]
Downloading:   1%|▏         | 14.7M/1.17G [00:03<05:11, 3.72MB/s]
Downloading:   2%|▏         | 17.8M/1.17G [00:03<04:07, 4.67MB/s]
Downloading:   2%|▏         | 19.9M/1.17G [00:03<03:34, 5.37MB/s]
Downloading:   2%|▏         | 23.1M/1.17G [00:03<03:00, 6.37MB/s]
Downloading:   2%|▏         | 26.2M/1.17G [00:04<02:37, 7.29MB/s]
Downloading:   3%|▎         | 31.5M/1.17G [00:04<01:55, 9.85MB/s]
Downloading:   2%|▏         | 25.2M/1.08G [00:03<02:27, 7.19MB/s]
Downloading:   3%|▎         | 33.6M/1.17G [00:04<02:08, 8.83MB/s]
Downloading:   3%|▎         | 35.7M/1.17G [00:05<02:48, 6.75MB/s]
Downloading:   3%|▎         | 34.6M/1.08G [00:04<01:48, 9.65MB/s]
Downloading:   3%|▎ 

CancelledError: ignored

In [ ]:
#api.download(S2B_MSIL1C_20180322T100019_N0206_R122_T33TUL_20180322T120713	,"Data/Inputs/")

### Finding our input data

We search for and list our input data. It should be in the form of zipped Sentinel2-files.  If we want to preprocess them for training, we will have to unzip them later, but if all we want to do is the superresolution itself, that will not be necessary.

In [ ]:
import glob
image_zips=(glob.glob("Data/Inputs/*.zip"))
image_zips

['Data/Inputs/S2A_MSIL1C_20190401T100031_N0207_R122_T33TWM_20190401T105727.zip']



---



### Application A: Training

The model is trained using patches which are randomly extracted from the input images. 
For the training process, three functions are relevant:

*   Preprocessing:
      * `readS2fromFile`: This function extracts a large number of random image patches from the input files and saves them in a new directory `train` or `train60`. 
      * `create_random`: This function indexes the image patches and randomly splits the index into a training and a testing dataset.
*   Training:
      * `train_DSen2`: Loads the image patches and trains a model from scratch or from a previously created weights file. 

We load these functions from the `/python` directory.


In [ ]:
import sys
sys.path.append("DSen2/python")
from create_patches import readS2fromFile
from create_random import create_random
from supres_train import train_DSen2
#from patches import recompose_images, OpenDataFilesTest, OpenDataFiles
#from DSen2Net import s2model

Using TensorFlow backend.


#### Extracting the Files

For creating the training files, the input files need to be unzipped. A bit annoying, but a simple task.
We extract all our files into the `Data/Training` folder

In [ ]:
import zipfile
for i in range(0, len(image_zips)):
  with zipfile.ZipFile(image_zips[i], 'r') as zip_ref:
      zip_ref.extractall("Data/Training")



In [ ]:
train_files=glob.glob("Data/Training/*SAFE",recursive=True)
train_files

['Data/Training/S2A_MSIL1C_20190401T100031_N0207_R122_T33TWM_20190401T105727.SAFE']

####  Setting the parameters



We have to set a number of parameters for the preprocessing of the data.
* **data_files**: The list of files from which tiles will be created. They will be processed iteratively.
* **training_path**: The directory which contains the unzipped images. This is also the directory into which outputs will be saved.
* **test_data**: If `true` stores test patches in a separate dir? 
* **roi_x_y**: A string which sets the region of interest (x_1,y_1,x_2,y_2) to extract as pixels locations on the 10m bands.
* **write_images**: If `true`, write PNG images into the output folder.
* **run_60**: If `true`, creates patches also from the 60m channels. This basically determines for which network (60m or 20m) we are creating the patches, and therefore we will also use this parameter later as a switch for training the model.
* **NR_CROP**: The number of image patches to be extracted per image
* **split_ratio**: The proportion of patches that goes into the testing partition.

In addition, there are certain parameters for the training itself.

* **deep**: Train a *very* deep network (VDSen2)? If `False`, trains a deep network.
* **model_name**: The name for the output model. This is the name under which the network weights and associated files will be output in a subdirectory `/network_data` withing the **training_path**
*   **SCALE**: A factor by which the raw reflectance values are divided by for "numerical stability".
*   **lr**: The base learning rate. When plateauing, it will be lowered by a callback.
* **n_epochs**: The number of epochs to train the model for.
* **resume_file**: This should be a string of the path to the weights we want to continue training from. Note that the weights file must match the model depth (as set by **deep**) and input shape (as set by **run_60**). 
If we want to train from scratch, we pass an empty string.


Here, we set **`run_60`** and **`deep`** to `True`, and we will resume from the file `"DSen2/models/s2_030_lr_1e-05.hdf5"` and tune it to a new model which will be saved into `Data/Training/network_data/GutesModell.hdf5`,

In [ ]:
##For the Preprocessing
data_files=train_files
training_path="Data/Training/"
test_data=False  
roi_x_y=""
write_images=True
run_60=True
NR_CROP=600
split_ratio=0.1

##For the Training
deep=False
model_name = "nov_model60"
SCALE = 2000
lr = 1e-4
n_epochs=4096


##Use this to train with the pre-trained DSen2/VDSen2 weights
if(deep):
  if(run_60):
    resume_file="DSen2/models/s2_034_lr_1e-04.hdf5"  
  else:
    resume_file="DSen2/models/s2_033_lr_1e-04.hdf5"  
else:
  if(run_60):
    resume_file="DSen2/models/s2_030_lr_1e-05.hdf5"
  else:
    resume_file="DSen2/models/s2_032_lr_1e-04.hdf5"

#uncomment to train from scratch or from a previous weight file
#resume_file= "Data/Training/network_data/nov_model60.hdf5"
#resume_file="Path/to/the/model.hdf5"

#### Creating the patches

Now we want to create the test patches. The function `readS2fromFile` iteratively reads in the Sentinel-2 images , extracts random test patches, and saves them into the `/training_path` where output folders `/train` or `/train_60` (if **`run_60`**) will be created. Afterwards, the input images themselves are strictly not necessary anymore, and we could delete them, but if we want to try the superresolution on them we should still keep them around.

In [ ]:
#for i in range(0, len(data_files)):
#  data_file=data_files[i]

readS2fromFile(data_files=data_files,
                test_data=test_data,
                roi_x_y=roi_x_y,
                save_prefix=training_path,
                write_images=write_images,
                run_60=run_60,
                NR_CROP=NR_CROP)

Data/Training/


Success.


#### Indexing and Splitting

The `create_random` will create an index of the training patches, but it does **not** do this by searching for the files we have created in the previous step. Rather, the number of patches must be given to the function. We can simply derive it from multiplying the parameter **`NR_CROP`** we set previously by the number of images.
It will create a file `val_index.npy` which will be saved within the specified folders subdirectories which we just created in the previous step, either `/train` or `/train_60` depending on whether we use the `run_60` switch or not (to ensure consistency, we just reuse the **`run_60`** we set previously in this script).

In [ ]:
###CREATE THE TRAIN INDEX AND SPLIT ###
create_random( size=len(data_files)*NR_CROP,ratio=split_ratio,out_path=training_path, run_60=run_60)

Full no of samples: 600
Validation samples: 60
Number of iterations: 61


function ClickConnect(){
    document.querySelector("colab-connect-button").click()
    console.log("Clicked on connect button"); 
}
setInterval(ClickConnect,60000)

####Training

Training is not complicated. We have set all of the parameters already. 

The inputs will be read from our training directory, specifically the subfolders `/train` or `/train60` (here, again, our **`run_60`** parameter comes into play), and split using `val_index.npy` we just put there.
The outputs will be saved within our training directory within a newly created subfolder `/network_data`.

In [ ]:
import time
start_time = time.time()
train_DSen2(model_name=model_name,path=training_path,run_60=run_60,deep=deep,resume_file=resume_file,SCALE=SCALE,lr=lr,n_epochs=n_epochs)
Deep_Time= time.time() - start_time


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Symbolic Model Created.
Model compiled.
Will resume from the weights DSen2/models/s2_030_lr_1e-05.hdf5
Changing the model Name to: nov_model60
Loading the training data...from...
Data/Training/


/tensorflow-1.15.2/python3.6/keras/callbacks/callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Loaded 600 patches for training.
Training starts...



Streaming output truncated to the last 5000 lines.
540/540 [==============================] - 4s 8ms/step - loss: 0.0049 - mean_squared_error: 5.7435e-05 - val_loss: 0.0052 - val_mean_squared_error: 6.2570e-05

Epoch 02847: val_loss did not improve from 0.00515
Epoch 2848/4096
540/540 [==============================] - 4s 8ms/step - loss: 0.0049 - mean_squared_error: 5.7428e-05 - val_loss: 0.0051 - val_mean_squared_error: 6.2548e-05

Epoch 02848: val_loss did not improve from 0.00515
Epoch 2849/4096
540/540 [==============================] - 4s 8ms/step - loss: 0.0049 - mean_squared_error: 5.7428e-05 - val_loss: 0.0051 - val_mean_squared_error: 6.2549e-05

Epoch 02849: val_loss did not improve from 0.00515
Epoch 2850/4096
540/540 [==============================] - 4s 8ms/step - loss: 0.0049 - mean_squared_error: 5.7428e-05 - val_loss: 0.0051 - val_mean_squared_error: 6.2538e-05

Epoch 02850: val_loss did not improve from 0.00515
Epoch 2851/4096
540/540 [==============================] 

Training done!

In [ ]:
print(deep_time)

NameError: ignored



---



### Application B: Prediction

For the superresolution we do not require much. Mostly all we need is a list of input images, the weights for our model, and the `DSen2_sl` function.

In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
import sys
sys.path.append("DSen2/python")
from DSen2_sl import DSen2_sl

Using TensorFlow backend.


#### Overview

The function uses reasonble defaults, which is the superresolution of the 20m bands into 10m bands, and combining them with the previous 10m bands into a stack which will be output as a GTiff. Nevertheless, there are a number of adjustments we can make by passing some parameters to the function:

* **input_files**: A list of input files.
* **output_dir** A directory into which the superresolved images will be output.
* **model_20_file**  A file to the weights of the 20m model.
* **model_60_file**: (Optional) If we provide also a file for the 60m model, the 60m bands will also be superresolved and added to the output. Otherwise, only the 20m band will be superresolved.

* **roi_lon_lat**: (Optional) Sets the region of interest to extract, WGS84, decimal notation. lon_1,lat_1,lon_2,lat_2.
* **roi_x_y**: (Optional) Sets the region of interest to extract as pixels locations on the 10m bands. Use this syntax: x_1,y_1,x_2,y_2. 

* **copy_original_bands**: (Optional) Copy the original 10m bands into the new, predicted file?
* **select_UTM**: (Optional) Manually select a UTM zone to use - otherwise the algorithm will select the UTM zone with the best coverage.
* **deep**: (Optional) Use a very deep network (VDSen2) instead of a deep network (DSen2)? Note that this must match the weights files provided.



#### Examples

Here are just some examples of the usage. Note that they will all be saved to the same output file, so executing multiple examples will result in the previous file being overwritten.

In [ ]:
#"DSen2/models/s2_032_lr_1e-04.hdf5"
#1,1,300,300
# roi_x_y=""
"Data/Training/network_data/slo_model.hdf5"
#2196,4392,6588,8784,10980

##### Basic Superresolution of just the 20m model
This is the basic usage. All we have to provide is an input image, the output directory to save to (will be created if it does not exist already) as well as the weights file to use.
We use the **`roi_x_y`** parameter to restrict ourselves to a small part of the image, to speed up the processing - this is usually not required.

In [ ]:
DSen2_sl(input_files=image_zips,
          output_dir="Data/Outputs/BasicDSen20/",
          model_20_file="DSen2/models/s2_032_lr_1e-04.hdf5",
           roi_x_y="1,1,6000,6000")


=========STARTING DSen2==========
roi_lon_lat:
roi_x_y:1,1,1500,1500
run_60:False
select_UTM:
deep:False
output_file_format:GTiff
copy_original_bands file:True
========SETTING UP MODEL=========
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Deep Symbolic Model 20 created and weights loaded from file:DSen2/models/s2_032_lr_1e-04.hdf5
Deep Symbolic Model 60 created and weights loaded from file: 
======STARTING PREDICTIONS=======
Processing data file:Data/Inputs/S2A_MSIL1C_20191008T100031_N0208_R122_T33TWM_20191008T110931.zip
Predicting using 20m Model

196/196 [==============================] - 3s 13ms/step
(6, 1500, 1500)
Writing to:Data/Outputs/BasicDSen20/S2A_MSIL1C_20191008T100031_N0208_R122_T33TWM_20191008T110931_supres.tif
---------Prediction Done---------
---------------------------------
Processing data file:Data/Inputs/S2A_MSIL1C_20190401T100031_N0207_R122_T33TWM_20190401T105727.zip
Predicting using 20m Model
196/196 [===========================

In [ ]:
 model_20_file="DSen2/models/s2_032_lr_1e-04.hdf5",
 model_60_file="DSen2/models/s2_030_lr_1e-05.hdf5",
 #my model
 "DSen2/models/s2_032_lr_1e-04.hdf5"

##### Superresolution of also the 60m bands

If we also want to superresolve the 60m bands, this is as easy as providing an additional weights file for the 60m model.

In [ ]:
DSen2_sl(input_files=image_zips,
          output_dir="Data/Outputs/BasicDSen60/",
          model_20_file="Data/Training/network_data/slo_model20.hdf5",
          model_60_file="Data/Training/network_data/slo_model60.hdf5",
          roi_x_y="1,1,10980,10980")

=========STARTING DSen2==========
roi_lon_lat:
roi_x_y:1,1,10980,10980
run_60:True
select_UTM:
deep:False
output_file_format:GTiff
copy_original_bands file:True
========SETTING UP MODEL=========
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Deep Symbolic Model 20 created and weights loaded from file:Data/Training/network_data/slo_model20.hdf5
Deep Symbolic Model 60 created and weights loaded from file: Data/Training/network_data/slo_model.hdf5
======STARTING PREDICTIONS=======
Processing data file:Data/Inputs/S2B_MSIL1C_20190913T100029_N0208_R122_T33TVL_20190913T134222.zip
Predicting using 60m Model

4356/4356 [==============================] - 58s 13ms/step
(2, 10980, 10980)
Predicting using 20m Model
9801/9801 [==============================] - 52s 5ms/step
(6, 10980, 10980)
Writing to:Data/Outputs/BasicDSen60/S2B_MSIL1C_20190913T100029_N0208_R122_T33TVL_20190913T134222_supres.tif
---------Prediction Done---------
---------------------------------


In [ ]:
DSen2_sl(input_files=image_zips,
          output_dir="Data/Outputs/ena/",
          model_20_file="Data/Training/network_data/slo_model20.hdf5",
          model_60_file="Data/Training/network_data/slo_model.hdf5",
          roi_x_y="1,1,5490,10980")
DSen2_sl(input_files=image_zips,
          output_dir="Data/Outputs/dva/",
          model_20_file="Data/Training/network_data/slo_model20.hdf5",
          model_60_file="Data/Training/network_data/slo_model.hdf5",
          roi_x_y="5490,1,10980,10980")

=========STARTING DSen2==========
roi_lon_lat:
roi_x_y:1,1,5490,10980
run_60:True
select_UTM:
deep:False
output_file_format:GTiff
copy_original_bands file:True
========SETTING UP MODEL=========
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Deep Symbolic Model 20 created and weights loaded from file:Data/Training/network_data/slo_model20.hdf5
Deep Symbolic Model 60 created and weights loaded from file: Data/Training/network_data/slo_model.hdf5
======STARTING PREDICTIONS=======
Processing data file:Data/Inputs/S2B_MSIL1C_20190913T100029_N0208_R122_T33TVL_20190913T134222.zip
Predicting using 60m Model

2178/2178 [==============================] - 32s 15ms/step
(2, 10980, 5490)
Predicting using 20m Model
4950/4950 [==============================] - 27s 5ms/step
(6, 10980, 5490)
Writing to:Data/Outputs/ena/S2B_MSIL1C_20190913T100029_N0208_R122_T33TVL_20190913T134222_supres.tif
---------Prediction Done---------
---------------------------------
=========STA

##### Superresolving using the Very Deep Model for 20m
To use the deep model, we switch the deep parameter to true, and also have to provide a weights file for the deep model.

In [ ]:
DSen2_sl(input_files=image_zips,
          output_dir="Data/Outputs/BasicVDSen20/",
          model_20_file="DSen2/models/s2_033_lr_1e-04.hdf5",
          deep=True,
          roi_x_y="1,1,300,300")

=========STARTING DSen2==========
roi_lon_lat:
roi_x_y:1,1,300,300
run_60:False
select_UTM:
deep:True
output_file_format:GTiff
copy_original_bands file:True
========SETTING UP MODEL=========


OSError: ignored

##### Superresolving using the Very Deep Model for 60m

And just as before, to use the 60m model we just provide the weights file.

In [ ]:
DSen2_sl(input_files=image_zips,
          output_dir="Data/Outputs/BasicVDSen60/",
          model_20_file="DSen2/models/s2_033_lr_1e-04.hdf5",
          model_60_file="DSen2/models/s2_034_lr_1e-04.hdf5",   
          #model_60_file="Data/Training/network_data/GutesModell.hdf5",  #Use this if we have created it in Part A.
          deep=True,
          roi_x_y="1,1,300,300")

=========STARTING DSen2==========
roi_lon_lat:
roi_x_y:1,1,300,300
run_60:True
select_UTM:
deep:True
output_file_format:GTiff
copy_original_bands file:True
========SETTING UP MODEL=========
Very Deep Symbolic Model 20 created and weights loaded from file:DSen2/models/s2_033_lr_1e-04.hdf5
Very Deep Symbolic Model 60 created and weights loaded from file: DSen2/models/s2_034_lr_1e-04.hdf5
======STARTING PREDICTIONS=======
Processing data file:Data/Inputs/S2B_MSIL1C_20190823T103029_N0208_R108_T32UMB_20190823T124349.zip
Predicting using 60m Model
4/4 [==============================] - 3s 667ms/step
(2, 300, 300)
Predicting using 20m Model
9/9 [==============================] - 2s 273ms/step
(6, 300, 300)
Writing to:Data/Outputs/BasicVDSen60/S2B_MSIL1C_20190823T103029_N0208_R108_T32UMB_20190823T124349_supres.tif
---------Prediction Done---------
---------------------------------
Processing data file:Data/Inputs/S2B_MSIL1C_20190823T103029_N0208_R108_T32UNA_20190823T124349.zip
Predicting using

##### Only 20m bands

If we are not interested in the original 10m bands (we already have them, after all) we can use the **`copy_original_bands`** switch to only return the bands we actually superresolved.

In [ ]:
DSen2_sl(input_files=image_zips,
          output_dir="Data/Outputs/OnlyDSen20/",
          model_20_file="DSen2/models/s2_032_lr_1e-04.hdf5",
          roi_x_y="1,1,300,300",copy_original_bands=False)

=========STARTING DSen2==========
roi_lon_lat:
roi_x_y:1,1,300,300
run_60:False
select_UTM:
deep:False
output_file_format:GTiff
copy_original_bands file:False
========SETTING UP MODEL=========
Deep Symbolic Model 20 created and weights loaded from file:DSen2/models/s2_032_lr_1e-04.hdf5
Deep Symbolic Model 60 created and weights loaded from file: 
======STARTING PREDICTIONS=======
Processing data file:Data/Inputs/S2B_MSIL1C_20190823T103029_N0208_R108_T32UMB_20190823T124349.zip
Predicting using 20m Model
9/9 [==============================] - 2s 172ms/step
(6, 300, 300)
Writing to:Data/Outputs/OnlyDSen20/S2B_MSIL1C_20190823T103029_N0208_R108_T32UMB_20190823T124349_supres.tif
---------Prediction Done---------
---------------------------------
Processing data file:Data/Inputs/S2B_MSIL1C_20190823T103029_N0208_R108_T32UNA_20190823T124349.zip
Predicting using 20m Model
9/9 [==============================] - 0s 7ms/step
(6, 300, 300)
Writing to:Data/Outputs/OnlyDSen20/S2B_MSIL1C_20190823T10302

##### Restricting the AOI
We can pass a string of X1, Y1, X2, X2 coordinates to **`roi_lon_lat`** to restrict the processing to a certain AOI. Of course, this mostly makes sense if the AOI overlaps all our input images, and will fail if one image has zero data within the AOI.

Here, we can see from the prediction time that our AOI contains a lot more data for the first image than the second, increasing the array size and prediction time.

In [ ]:
DSen2_sl(input_files=image_zips,
          output_dir="Data/Outputs/AOI_lon_lat/",
          model_20_file="DSen2/models/s2_032_lr_1e-04.hdf5",
          model_60_file="DSen2/models/s2_030_lr_1e-05.hdf5",
         roi_lon_lat="9,50.525986633828595,9.115343725999678,50.6521965147419")

=========STARTING DSen2==========
roi_lon_lat:9,50.525986633828595,9.115343725999678,50.6521965147419
roi_x_y:
run_60:True
select_UTM:
deep:False
output_file_format:GTiff
copy_original_bands file:True
========SETTING UP MODEL=========
Deep Symbolic Model 20 created and weights loaded from file:DSen2/models/s2_032_lr_1e-04.hdf5
Deep Symbolic Model 60 created and weights loaded from file: DSen2/models/s2_030_lr_1e-05.hdf5
======STARTING PREDICTIONS=======
Processing data file:Data/Inputs/S2B_MSIL1C_20190823T103029_N0208_R108_T32UMB_20190823T124349.zip
Predicting using 60m Model
45/45 [==============================] - 3s 74ms/step
(2, 1404, 816)
Predicting using 20m Model
104/104 [==============================] - 3s 30ms/step
(6, 1404, 816)
Writing to:Data/Outputs/AOI_lon_lat/S2B_MSIL1C_20190823T103029_N0208_R108_T32UMB_20190823T124349_supres.tif
---------Prediction Done---------
---------------------------------
Processing data file:Data/Inputs/S2B_MSIL1C_20190823T103029_N0208_R108_T32

#### Timing the Models

To see how much faster the DSen2 is compared to its deeper counterpart VDSen2, we roughly time the prediction twice - once for a small AOI, once for a medium AOI and once for a large AOI.

##### Small AOI

In [ ]:
%%capture
import time
start_time = time.time()
DSen2_sl(input_files=image_zips,
          output_dir="Data/Outputs/TimingTest/",
          model_20_file="DSen2/models/s2_032_lr_1e-04.hdf5",
          model_60_file="DSen2/models/s2_030_lr_1e-05.hdf5",
          roi_x_y="1,1,300,300")
Deep_Time= time.time() - start_time

start_time = time.time()
DSen2_sl(input_files=image_zips,
          output_dir="Data/Outputs/TimingTest/",
          model_20_file="DSen2/models/s2_033_lr_1e-04.hdf5",
          model_60_file="DSen2/models/s2_034_lr_1e-04.hdf5",   
          deep=True,
          roi_x_y="1,1,300,300")
Very_Deep_Time= time.time() - start_time

In [ ]:
print("DSen2 Time: " + str(Deep_Time)+"\n VDSen2 Time: "+str(Very_Deep_Time))

DSen2 Time: 24.032052278518677
 VDSen2 Time: 31.71060013771057


Not a massive difference.

##### Medium AOI

In [ ]:
%%capture
import time
start_time = time.time()
DSen2_sl(input_files=image_zips,
          output_dir="Data/Outputs/TimingTest/",
          model_20_file="DSen2/models/s2_032_lr_1e-04.hdf5",
          model_60_file="DSen2/models/s2_030_lr_1e-05.hdf5",
          roi_x_y="1,1,1000,1000")
Deep_Time= time.time() - start_time

start_time = time.time()
DSen2_sl(input_files=image_zips,
          output_dir="Data/Outputs/TimingTest/",
          model_20_file="DSen2/models/s2_033_lr_1e-04.hdf5",
          model_60_file="DSen2/models/s2_034_lr_1e-04.hdf5",   
          deep=True,
          roi_x_y="1,1,1000,1000")
Very_Deep_Time= time.time() - start_time

In [ ]:
print("DSen2 Time: " + str(Deep_Time)+"\n VDSen2 Time: "+str(Very_Deep_Time))

DSen2 Time: 31.127756595611572
 VDSen2 Time: 59.742024421691895


##### Large AOI

In [ ]:
%%capture
import time
start_time = time.time()
DSen2_sl(input_files=image_zips,
          output_dir="Data/Outputs/TimingTest/",
          model_20_file="DSen2/models/s2_032_lr_1e-04.hdf5",
          model_60_file="DSen2/models/s2_030_lr_1e-05.hdf5",
          roi_x_y="1,1,4000,4000")
Deep_Time= time.time() - start_time

start_time = time.time()
DSen2_sl(input_files=image_zips,
          output_dir="Data/Outputs/TimingTest/",
          model_20_file="DSen2/models/s2_033_lr_1e-04.hdf5",
          model_60_file="DSen2/models/s2_034_lr_1e-04.hdf5",   
          deep=True,
          roi_x_y="1,1,4000,4000")
Very_Deep_Time= time.time() - start_time

In [ ]:
print("DSen2 Time: " + str(Deep_Time)+"\n VDSen2 Time: "+str(Very_Deep_Time))

DSen2 Time: 246.17463541030884
 VDSen2 Time: 580.7024714946747


Now the difference really shows!